In [18]:
import duckdb

cn = duckdb.connect(database=':memory:')

sql = """
    CREATE or replace view v_c2_detail as 
    select Number as stroke_number
        ,"Time (seconds)" as total_time
        ,"Distance (meters)" as total_distance
        ,"Pace (seconds)" as pace
        ,watts
        ,"Stroke Rate" as stroke_rate
        ,season as season_year
        ,"date" as workout_date
        ,machine_type
        ,strftime(
            TIMESTAMP '1970-01-01 00:00:00'
            + CAST(round("pace") AS INTEGER) * INTERVAL 1 SECOND,
            '%M:%S'
        ) AS total_time_mm_ss
        ,printf(
            '%d:%02d.%02d',
            CAST(floor(CAST(round("pace" * 100) AS INTEGER) / 6000) AS INTEGER),            -- minutes
            CAST(floor((CAST(round("pace" * 100) AS INTEGER) % 6000) / 100) AS INTEGER),    -- seconds
            CAST(CAST(round("pace" * 100) AS INTEGER) % 100 AS INTEGER)                     -- hundredths
        ) AS pace_mm_ss_ff
        ,CAST(regexp_extract(filename, '([0-9]+)\\.csv$', 1) AS BIGINT) AS log_id
    from read_csv_auto('~/concept2/workouts/*detail*.csv', filename=true)
"""


cn.execute(sql)

In [21]:
sql = """
    from v_c2_detail
    where workout_date >= '2025-12-23'
    --limit 50
"""

cn.sql(sql)

┌───────────────┬────────────┬────────────────┬────────┬───────┬─────────────┬─────────────┬──────────────┬──────────────┬──────────────────┬───────────────┬───────────┐
│ stroke_number │ total_time │ total_distance │  pace  │ Watts │ stroke_rate │ season_year │ workout_date │ machine_type │ total_time_mm_ss │ pace_mm_ss_ff │  log_id   │
│     int64     │   double   │     double     │ double │ int64 │    int64    │    int64    │     date     │   varchar    │     varchar      │    varchar    │   int64   │
├───────────────┼────────────┼────────────────┼────────┼───────┼─────────────┼─────────────┼──────────────┼──────────────┼──────────────────┼───────────────┼───────────┤
│             1 │        0.3 │            1.0 │    0.0 │  NULL │        NULL │        2026 │ 2025-12-23   │ rowerg       │ 00:00            │ 0:00.00       │ 110190940 │
│             2 │        2.3 │            8.5 │  151.0 │   102 │        NULL │        2026 │ 2025-12-23   │ rowerg       │ 02:31            │ 2:31.00 